In [1]:
pip install albumentations

In [9]:
import albumentations as A
import cv2
import pandas as pd
import os
import numpy as np

def augment_image_and_landmarks(image_path, landmarks, image_dir, output_dir, output_label_dir, augmentation_transform):
    """Augments an image and its corresponding landmarks."""
    full_image_path = os.path.join(image_dir, image_path)
    print(f"Processing image: {full_image_path}") #add print statement.
    if os.path.exists(full_image_path):
        img = cv2.imread(full_image_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            height, width, _ = img.shape

            # Convert landmarks to albumentations keypoints format (with angle and scale)
            keypoints = []
            for i in range(1, 20):
                x = landmarks[f'{i}_x']
                y = landmarks[f'{i}_y']
                keypoints.append((x, y, i - 1, 0, 1))  # (x, y, class_id, angle, scale) - angle is 0, scale is 1

            # Convert keypoints list to NumPy array
            keypoints = np.array(keypoints)

            # Apply augmentations
            augmented = augmentation_transform(image=img, keypoints=keypoints)
            augmented_image = augmented['image']
            augmented_keypoints = augmented['keypoints']

            # Save augmented image
            output_image_path = os.path.join(output_dir, f"aug_{image_path}")
            cv2.imwrite(output_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            # Save augmented labels in YOLO format
            image_name = os.path.splitext(os.path.basename(image_path))[0]
            label_file_path = os.path.join(output_label_dir, f"aug_{image_name}.txt")

            with open(label_file_path, 'w') as f:
                for x, y, class_id, _, _ in augmented_keypoints: #add _ to ignore angle and scale
                    x_center = x / width
                    y_center = y / height
                    bbox_width = 10 / width #adjust
                    bbox_height = 10 / height #adjust

                    if bbox_width <= 0 or bbox_height <= 0 or x_center <=0 or y_center <=0 or x_center >=1 or y_center >=1:
                        print(f"Warning: Invalid augmented bounding box for image {image_path}, Landmark {class_id}. Skipping.")
                        continue

                    f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

        else:
            print(f"Error: Could not read image {full_image_path}")
            return #stop the function, since there is no image.
    else:
        print(f"Error: Image file not found: {full_image_path}")
        return #stop the function, since there is no image.

# Example Usage:
train_data = pd.read_csv("/content/drive/MyDrive/FYP/train_cepha.csv")
image_dir = "/content/drive/MyDrive/FYP/cepha400"
output_image_dir = "augmented_images"
output_label_dir = "augmented_labels"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

augmentation_transform = A.Compose([
    A.Rotate(limit=15, p=0.7),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
])

for index, row in train_data.iterrows():
    image_path = row['image_path']
    augment_image_and_landmarks(image_path, row, image_dir, output_image_dir, output_label_dir, augmentation_transform)

print("Augmentation complete.")

Processing image: /content/drive/MyDrive/FYP/cepha400/017.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/121.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/033.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/093.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/052.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/045.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/021.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/029.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/080.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/030.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/147.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/019.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/131.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/127.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/148.jpg
Processing image: /content/drive/MyDrive/FYP/cepha400/098.jpg
Processi

In [ ]:
import os
import shutil
import pandas as pd

original_image_dir = "cepha_400"
augmented_image_dir = "augmented_images"
original_label_dir = "labels" #original labels, if they exist.
augmented_label_dir = "augmented_labels"
combined_image_dir = "combined_images"
combined_label_dir = "combined_labels"

os.makedirs(combined_image_dir, exist_ok=True)
os.makedirs(combined_label_dir, exist_ok=True)

# Copy original images and labels
for filename in os.listdir(original_image_dir):
    shutil.copy(os.path.join(original_image_dir, filename), os.path.join(combined_image_dir, filename))
    label_filename = os.path.splitext(filename)[0] + ".txt"
    if os.path.exists(os.path.join(original_label_dir, label_filename)):
        shutil.copy(os.path.join(original_label_dir, label_filename), os.path.join(combined_label_dir, label_filename))
    else:
        print(f"Warning: original Label file {label_filename} not found.")

# Copy augmented images and labels
for filename in os.listdir(augmented_image_dir):
    shutil.copy(os.path.join(augmented_image_dir, filename), os.path.join(combined_image_dir, filename))
    label_filename = os.path.splitext(filename)[0] + ".txt"
    shutil.copy(os.path.join(augmented_label_dir, label_filename), os.path.join(combined_label_dir, label_filename))

print("Combination complete.")